In [ ]:
!pip install pymysql

In [ ]:
import pymysql

# mysql 연동
1. DB 서버와의 연결
    - 서버의 주소
    - 아이디
    - 비밀번호(설정을 안 한 경우에는 적지 않는다.)
    - Database
    - Port
2. Cursor를 생성 
    - sql 서버와 python사이에 있는 가상의 공간
3. sql 쿼리문을 작성 
4. Cursor를 이용하여 sql쿼리문을 sql 서버에 보내준다. (execute)
5. Select문 경우에는 결과 값을 Cursor에서 받아온다. 
6. insert, update, delete문의 경우에는 데이터가 수정이 됨으로 sql 서버에 데이터를 변경(commit)
7. sql 서버와의 연결을 종료

In [ ]:
# 서버와의 연결 
_db = pymysql.connect(
    host = '127.0.0.1',   # localhost
    user = 'root', 
    password = '1234', 
    database= 'ezen', 
    port = 3306
)

In [ ]:
# Cursor 생성 
# cursor() : 인자값을 넣지 않으면 결과값을 받아올때 tuple의 형태로 데이터를 가져온다. 
cursor = _db.cursor(pymysql.cursors.DictCursor)

In [ ]:
# 테이블을 생성하는 sql 쿼리문을 작성 
sql = """
    create 
    table 
    user(
        id varchar(32) primary key, 
        password varchar(32) not null, 
        name varchar(32) not null, 
        gender int
    );
"""
# cursor에 있는 execute()함수를 이용하여 쿼리문을 실행
cursor.execute(sql)

In [ ]:
# insert문을 이용하여 데이터를 삽입
sql = """
    insert 
    into 
    user(
        id, 
        password, 
        name
    )
    values 
    (
        'test', 
        '1234', 
        'kim'
    )
"""

cursor.execute(sql)

In [ ]:
_db.commit()

In [ ]:
# 외부의 데이터베이스 서버에 접속

_db2 = pymysql.connect(
    host = 'darkpreist.iptime.org', 
    user = 'ubion', 
    password = '1234', 
    database= 'ubion', 
    port = 3306
)

In [ ]:
cursor = _db2.cursor(pymysql.cursors.DictCursor)

In [ ]:
sql = """
    SELECT 
    `Region` as '대륙', 
    `Country` as '국가', 
    `Sales Channel` as '채널', 
    `Units Sold` as '개수', 
    `Unit Cost` as '원가', 
    `Unit Price` as '단가'
    FROM 
    `sales records`
"""

cursor.execute(sql)

In [ ]:
# cursor 공간에서 데이터를 불러온다. 
data = cursor.fetchall()

In [ ]:
data

In [ ]:
import pandas as pd

In [ ]:
df = pd.DataFrame(data)
df

# 데이터의 결합
1. Union 결합 ( 데이터 간의 단순한 결합 )
    - 단순하게 행을 결합, 열을 결합 
    - sql
        - table UNION table2
    - pandas 
        - concat([df, df2], axis={0|1})
2. Join 결합 ( 특정한 조건에 맞춰서 열을 결합 )
    - 특정한 컬럼의 데이터를 기준으로 하여 해당하는 값들이 일치한다면 열을 결합
    - sql
        - table join table2 on 조건식
    - pandas 
        - merge(df, df2, on = 조건식, how = 결합방식)

In [ ]:
sql = """
    select 
    * 
    from 
    tran_1
"""
cursor.execute(sql)
data1 = cursor.fetchall()
df = pd.DataFrame(data1)

In [ ]:
df

In [ ]:
sql = """
    select 
    * 
    from tran_2
"""
cursor.execute(sql)
data2 =cursor.fetchall()
df2 = pd.DataFrame(data2)
df2

In [ ]:
# sql 쿼리문을 이용하여 데이터를 단순 행 결합
sql = """
    select * from tran_1 
    union 
    select * from tran_2;
"""
cursor.execute(sql)
data3 = cursor.fetchall()
df3 = pd.DataFrame(data3)
df3

In [ ]:
# pandas를 이용하여 데이터프레임을 단순 행 결합 
pd.concat([df, df2], axis=0, ignore_index=True)

In [ ]:
# 인자값으로 쿼리문을 입력하면 결과를 데이터프레임으로 되돌려주는 함수
def sql_load(sql):
    cursor.execute(sql)
    data = cursor.fetchall()
    result = pd.DataFrame(data)
    return result

In [ ]:
sql = """
    select 
    * 
    from 
    emp
"""
emp = sql_load(sql)
emp

In [ ]:
sql = """
    select 
    * 
    from 
    dept
"""
dept = sql_load(sql)
dept

In [ ]:
# join 결합 sql
sql = """
    select 
    *
    from 
    emp 
    right join 
    dept 
    on 
    emp.DEPTNO = dept.DEPTNO
"""
sql_load(sql)

In [ ]:
# pandas를 이용하여 join 결합 (merge)
pd.merge(emp, dept, on = 'DEPTNO', how = 'right')

In [ ]:
class Mysql:
    # 생성자 함수 : class가 생성 될때 최초로 한번만 실행이 되는 함수을 의미한다. 초기화 함수라고 부르기도 한다. 
    def __init__(self, _host, _user, _pass, _db, _port):
        self.host = _host
        self.user = _user
        self.password = _pass
        self.db = _db
        self.port = _port
    
    # sql 쿼리문을 입력 값으로 받아서 쿼리문을 실행하고 결과값을 되돌려주는 함수 
    def sql_load(self, _sql, _value = []):
        # 데이터베이스 서버와의 연결
        _db = pymysql.connect(
            host = self.host, 
            user = self.user, 
            password = self.password, 
            database = self.db, 
            port = self.port
        )
        # Cursor 생성
        cursor = _db.cursor(pymysql.cursors.DictCursor)

        # sql 쿼리문을 실행 
        cursor.execute(_sql, _value)

        # _sql이 시작이 select로 시작하는 경우
        if _sql.strip().lower().startswith('select'):
            result = cursor.fetchall()
        # 아닌 경우
        else:
            _db.commit()
            result = 'Query OK'
        
        # 데이터베이스와의 연결을 종료
        _db.close()
        
        return result


In [ ]:
sql = """
    insert 
    * 
    from 
    emp
"""
# sql.strip().lower()[0:6] == 'select'
sql.strip().lower().startswith('select')